In [ ]:
from cpe_dict_loader import *
from registry_query import *
from cpe_similarity import *
import json,random

In [ ]:
software_inventory = get_installed_software()
cpe_list = load_cpe_processed_json()

In [ ]:
with open("test_cpe_set.json","r") as file:
    test_set = json.load(file)
    file.close()

In [ ]:
with open("NVD_vendors.txt","r") as file:
    vendors_list = file.read().splitlines()
    file.close()

In [ ]:
#******************ENSEMBLE MATCHING******************

all_matches = []

for software in test_set:
    best_match = {"CPE":None,"Similarity":0}

    if software.get("Vendor") and software.get("Name"):  
        vendor_importance = assign_importance(software.get("Vendor"),vendors_list)
        name_importance = assign_importance(software.get("Name"),vendors_list)
        
    else:
        continue

    for cpe in cpe_list:         
        score = ensemble_similarity(cpe[1],software.get("Vendor"),software.get("Name"),software.get("Version"),vendor_importance,name_importance)

        if score > best_match["Similarity"] and cpe[1][2] == "a" and cpe[1][10] in ["windows","*"]:
            best_match["CPE"] = cpe[0]
            best_match["Similarity"] = score

    all_matches.append({"Software":software,"Best_match":best_match})

    if all_matches[-1]["Best_match"]["Similarity"] > 0.0: print(all_matches[-1])

In [ ]:
from gensim.models import FastText as ft
from gensim.models.fasttext import load_facebook_model

vendors_mapped = list(map(lambda x:[x],vendors_list))

model = load_facebook_model("crime-and-punishment.bin")
"""
model = ft(vector_size = 4,window = 1,min_count = 1)
model.build_vocab(corpus_iterable = vendors_mapped)
model.train(corpus_iterable = vendors_mapped,total_examples = len(vendors_mapped),epochs=10)"""

In [ ]:
model.wv.most_similar("mifcrosfot")

In [ ]:
#lookup for vendors DONE
#fasttext most_similar 
#confluence page